## Refactor some python dummy controller




### Instantiation

In [ ]:
import uuid
import json

from sqlalchemy import create_engine, String, Column, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///alice.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

class UserModel(Base):
    __tablename__ = 'User'
    id_ = Column(String(36), primary_key=True)
    client = Column(String, primary_key=False)
    name = Column(String, nullable=False)
    age = Column(Integer, nullable=False)

    def __init__(self, client, name, age):
        self.id_ = str(uuid.uuid4())
        self.client = client
        self.name = name
        self.age = age

Base.metadata.create_all(engine)

                         
class FakeEventBus:

    @staticmethod
    def publish(event):
        print(f"event sent: {event.__class__.__name__} {event.id_}")                        

### Create User Controller

This must persist a new user and send and domain event UserCreated to a fake bus.

In [ ]:
class UserCreated:

    def __init__(self, id_, client, name, age):
        self.id_ = id_
        self.client = client
        self.name = name
        self.age = age


def post_user(client_name, user_name, user_age):
    print("post_user")
    try:
        
        user_model = (
            session.query(UserModel)
            .filter(UserModel.client == client_name)
            .filter(UserModel.name == user_name)
            .filter(UserModel.age == user_age)
            .first()
        )
        
        if user_model:
            return 404, {"error": "user already exist"}
        
        user_model = UserModel(client_name, user_name, user_age)
        session.add(user_model)
        session.commit()

        user_created = UserCreated(user_model.id_, user_model.client, user_model.name, user_model.age)
        FakeEventBus.publish(user_created)
    except Exception as e:
        print(e)
        return 500, {"error": "error create user"}

    return 200, {"id": user_model.id_}

In [ ]:
client_name = "acme"
user_name = "Coyote"
user_age = 19
response = post_user(client_name, user_name, user_age)
print(response)

### Retrieve All Users Controller

This must retrieve a list of users

In [ ]:
def get_users(client_name):
    print("get_users")
    users = []
    try:
        
        user_models = (
            session.query(UserModel)
            .filter(UserModel.client == client_name)
            .all()
        )
        
        if not user_models:
            return 200, users
        
        users = [{user_model.id_, user_model.client, user_model.name, user_model.age} for user_model in user_models]
        
    except Exception as e:
        print(e)
        return 500, {"error": "error retrieving user"}

    return 200, users

In [ ]:
response = get_users(client_name)
print(response)
